In [16]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Load the Excel file
file_path = 'Project_Dataset.xlsx'
product_data = pd.read_excel(file_path, sheet_name='Product Recommendation Data')

# Display the first few rows of the dataframe
product_data.head()


,TransactionNo,Date,ProductID,Description,Price,Quantity,CustomerID,Country,Unnamed: 8,https://www.kaggle.com/datasets/gabrielramos87/an-online-shop-business
0,581482,2019-09-12,22485,Set Of 2 Wooden Market Crates,21.47,12,17490,United Kingdom,NaN,NaN
1,581475,2019-09-12,22596,Christmas Star Wish List Chalkboard,10.65,36,13069,United Kingdom,NaN,NaN
2,581475,2019-09-12,23235,Storage Tin Vintage Leaf,11.53,12,13069,United Kingdom,NaN,NaN
3,581475,2019-09-12,23272,Tree T-Light Holder Willie Winkie,10.65,12,13069,United Kingdom,NaN,NaN
4,581475,2019-09-12,23239,Set Of 4 Knick Knack Tins Poppies,11.94,6,13069,United Kingdom,NaN,NaN


In [17]:
# Fill missing descriptions with an empty string
product_data['Description'] = product_data['Description'].fillna('')

In [18]:
# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

In [19]:
# Fit and transform the descriptions to a TF-IDF matrix
tfidf_matrix = tfidf_vectorizer.fit_transform(product_data['Description'])

In [20]:
# Compute cosine similarity between items
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [21]:
# Create a user-item interaction matrix
user_item_matrix = product_data.pivot_table(index='CustomerID', columns='ProductID', values='Quantity', fill_value=0)

In [22]:
# Transpose the matrix for item-based collaborative filtering
item_user_matrix = user_item_matrix.T

In [23]:
# Initialize the Nearest Neighbors model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')

In [24]:
# Fit the model
model_knn.fit(item_user_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [25]:
# Define a function to get content-based recommendations
def get_content_based_recommendations(product_id, cosine_sim=cosine_sim):
    idx = product_data[product_data['ProductID'] == product_id].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    product_indices = [i[0] for i in sim_scores]
    return product_data.iloc[product_indices]

# Define a function to get item-based recommendations
def get_item_based_recommendations(product_id, model_knn=model_knn, item_user_matrix=item_user_matrix):
    idx = item_user_matrix.index.get_loc(product_id)
    distances, indices = model_knn.kneighbors(item_user_matrix.iloc[idx, :].values.reshape(1, -1), n_neighbors=11)
    similar_indices = indices.flatten()[1:]
    similar_products = item_user_matrix.index[similar_indices]
    return product_data[product_data['ProductID'].isin(similar_products)]

# Define a function to get hybrid recommendations
def get_hybrid_recommendations(product_id):
    content_recommendations = get_content_based_recommendations(product_id)
    item_recommendations = get_item_based_recommendations(product_id)
    hybrid_recommendations = pd.concat([content_recommendations, item_recommendations]).drop_duplicates().head(10)
    return hybrid_recommendations


In [26]:
# Generate recommendations for a sample product
sample_product_id = 23495


In [27]:
# Get content-based recommendations
content_recs = get_content_based_recommendations(sample_product_id)
content_recs

,TransactionNo,Date,ProductID,Description,Price,Quantity,CustomerID,Country,Unnamed: 8,https://www.kaggle.com/datasets/gabrielramos87/an-online-shop-business
633,581492,2019-09-12,22720,Set Of 3 Cake Tins Pantry Design,6.19,2,15492,United Kingdom,NaN,NaN
426,581492,2019-09-12,22196,Small Heart Measuring Spoons,7.24,11,15492,United Kingdom,NaN,NaN
425,581492,2019-09-12,22195,Large Heart Measuring Spoons,7.24,1,15492,United Kingdom,NaN,NaN
704,581492,2019-09-12,22989,Set 2 Pantry Design Tea Towels,6.19,2,15492,United Kingdom,NaN,NaN
768,581492,2019-09-12,23266,Set Of 3 Wooden Stocking Decoration,6.19,2,15492,United Kingdom,NaN,NaN
784,581492,2019-09-12,23307,Set Of 60 Pantry Design Cake Cases,6.19,7,15492,United Kingdom,NaN,NaN
767,581492,2019-09-12,23265,Set Of 3 Wooden Tree Decorations,6.19,3,15492,United Kingdom,NaN,NaN
706,581492,2019-09-12,22993,Set Of 4 Pantry Jelly Moulds,6.19,3,15492,United Kingdom,NaN,NaN
766,581492,2019-09-12,23264,Set Of 3 Wooden Sleigh Decorations,6.19,3,15492,United Kingdom,NaN,NaN
88,581486,2019-09-12,22561,Wooden School Colouring Set,6.04,12,17001,United Kingdom,NaN,NaN


In [28]:
# Get item-based recommendations
item_recs = get_item_based_recommendations(sample_product_id)
item_recs

,TransactionNo,Date,ProductID,Description,Price,Quantity,CustomerID,Country,Unnamed: 8,https://www.kaggle.com/datasets/gabrielramos87/an-online-shop-business
746,581492,2019-09-12,23183,Mother's Kitchen Spoon Rest,6.19,3,15492,United Kingdom,NaN,NaN
748,581492,2019-09-12,23188,Vintage 2 Metre Folding Ruler,6.19,1,15492,United Kingdom,NaN,NaN
749,581492,2019-09-12,23191,Bundle Of 3 Retro Note Books,6.19,1,15492,United Kingdom,NaN,NaN
750,581492,2019-09-12,23194,Gymkhana Treasure Book Box,6.19,1,15492,United Kingdom,NaN,NaN
751,581492,2019-09-12,23196,Vintage Leaf Magnetic Notepad,6.19,1,15492,United Kingdom,NaN,NaN
752,581492,2019-09-12,23197,Sketchbook Magnetic Shopping List,6.19,2,15492,United Kingdom,NaN,NaN
753,581492,2019-09-12,23198,Pantry Magnetic Shopping List,6.19,2,15492,United Kingdom,NaN,NaN
757,581492,2019-09-12,23213,Star Wreath Decoration With Bell,6.19,7,15492,United Kingdom,NaN,NaN
817,581492,2019-09-12,23199,Jumbo Bag Apples,6.19,2,15492,United Kingdom,NaN,NaN
818,581492,2019-09-12,23200,Jumbo Bag Pears,6.19,1,15492,United Kingdom,NaN,NaN


In [29]:
# Get hybrid recommendations
hybrid_recs = get_hybrid_recommendations(sample_product_id)
hybrid_recs [-9:]

,TransactionNo,Date,ProductID,Description,Price,Quantity,CustomerID,Country,Unnamed: 8,https://www.kaggle.com/datasets/gabrielramos87/an-online-shop-business
426,581492,2019-09-12,22196,Small Heart Measuring Spoons,7.24,11,15492,United Kingdom,NaN,NaN
425,581492,2019-09-12,22195,Large Heart Measuring Spoons,7.24,1,15492,United Kingdom,NaN,NaN
704,581492,2019-09-12,22989,Set 2 Pantry Design Tea Towels,6.19,2,15492,United Kingdom,NaN,NaN
768,581492,2019-09-12,23266,Set Of 3 Wooden Stocking Decoration,6.19,2,15492,United Kingdom,NaN,NaN
784,581492,2019-09-12,23307,Set Of 60 Pantry Design Cake Cases,6.19,7,15492,United Kingdom,NaN,NaN
767,581492,2019-09-12,23265,Set Of 3 Wooden Tree Decorations,6.19,3,15492,United Kingdom,NaN,NaN
706,581492,2019-09-12,22993,Set Of 4 Pantry Jelly Moulds,6.19,3,15492,United Kingdom,NaN,NaN
766,581492,2019-09-12,23264,Set Of 3 Wooden Sleigh Decorations,6.19,3,15492,United Kingdom,NaN,NaN
88,581486,2019-09-12,22561,Wooden School Colouring Set,6.04,12,17001,United Kingdom,NaN,NaN


In [30]:
#content_recs, item_recs, hybrid_recs